Deploying the agent to AgentCore Runtime

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import utils
boto_session = Session()
region = boto_session.region_name
agentcore_gateway_iam_role = utils.create_agentcore_role("lambdagateway")

agentcore_runtime = Runtime()
agent_name = "mortgage_assistant"
response = agentcore_runtime.configure(
    entrypoint="mortgage_agent_runtime_gw.py",
    execution_role=agentcore_gateway_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Launching agent to AgentCore Runtime

In [ ]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

Launching agent to AgentCore Runtime

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
print(status)

In [ ]:
import json
try:
    invoke_response = agentcore_runtime.invoke({"prompt": "what is the credit score for customerid 1234"})
    response_bytes = invoke_response["response"]
    response_str = response_bytes[0].decode('utf-8')  # Convert bytes to string
    response_json = json.loads(response_str)  # Parse JSON
    text_value = response_json["result"]["content"][0]["text"]  # Extract text
    print(text_value)
except Exception as e:
     print(invoke_response["response"])